In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint

In [ ]:
# make page
paper_size = '8x8 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 490
thetas = np.linspace(0, np.pi*2, n_lines)
radii = np.linspace(12, 12, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 10.7
t = np.linspace(0, t_max, 121)
a = 0.2
b = 0.95
c = - 0.4
d = -0.1

all_polys = Polygon()

break_dist = 0.05

lines = []
lfs = MultiLineString()
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



t_max = 10.7
t = np.linspace(0, t_max, 121)
a = 0.2
b = 0.95
c = - 0.6
d = -0.2

all_polys = Polygon()
break_dist = 0.05

lines = []
lfs = MultiLineString()
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)

lbs1 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs1 = gp.make_like(gp.merge_LineStrings(lbs1), drawbox)
lbs1 = sa.rotate(lbs1, 180)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
sk.geometry(lbs1)
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    
for ls in lbs1:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
cam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_pendown = 10
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.5)